Pre-process Hospital Discharge Data Public Use Data File dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

REPOSITORY_PATH = "/home/ramongonze/phd/privacy-ml" # privacy-ml repository path

In [2]:
cols = [
    "TYPE_OF_ADMISSION",
    "PAT_ZIP",
    "PAT_COUNTRY",
    "PAT_COUNTY",
    "PAT_STATUS",
    "SEX_CODE",
    "RACE",
    "ADMIT_WEEKDAY",
    "PAT_AGE",
    "TOTAL_CHARGES",
    "PRINC_DIAG_CODE"
]
df = pd.read_csv(
    os.path.join(REPOSITORY_PATH, "data/hospitals/hospitals_original.txt"),
    sep="\t",
    header=0,
    na_values=["`"],  # optional: strings that should become NaN
    encoding="utf-8",
    usecols=cols
)
display(df)

,TYPE_OF_ADMISSION,PAT_ZIP,PAT_COUNTRY,PAT_COUNTY,PAT_STATUS,SEX_CODE,RACE,ADMIT_WEEKDAY,PAT_AGE,TOTAL_CHARGES,PRINC_DIAG_CODE
0,3.0,79925.0,US,141.0,1.0,M,4.0,2.0,16.0,12038.73,N401
1,3.0,79912.0,US,141.0,6.0,M,4.0,2.0,16.0,54044.11,M1711
2,3.0,79928.0,US,141.0,6.0,M,5.0,2.0,17.0,59416.71,M1712
3,3.0,79835.0,US,141.0,6.0,F,5.0,2.0,18.0,42380.01,M1711
4,3.0,79901.0,US,141.0,6.0,F,5.0,3.0,15.0,39150.56,M1711
...,...,...,...,...,...,...,...,...,...,...,...
788838,4.0,76273.0,US,181.0,1.0,F,4.0,5.0,0.0,3719.27,Z3801
788839,1.0,74733.0,US,NaN,1.0,F,4.0,2.0,15.0,67745.88,A4159
788840,1.0,NaN,US,181.0,1.0,NaN,4.0,1.0,24.0,38787.12,J441
788841,3.0,75090.0,US,181.0,1.0,F,5.0,5.0,8.0,12264.18,O80


In [3]:
# Drop nan values
df = df.dropna()
print(f"# rows after dropping nan values: {len(df)}")

# rows after dropping nan values: 699716


In [4]:
print("Domain sizes")
for att in cols:
    values = df[att].unique().tolist()
    print(f"{att}: {len(values)}")
    
    if len(values) < 300:
        print(f"values: {set(values)}\n")

Domain sizes
TYPE_OF_ADMISSION: 6
values: {1.0, 2.0, 3.0, 4.0, 5.0, 9.0}

PAT_ZIP: 1614
PAT_COUNTRY: 1
values: {'US'}

PAT_COUNTY: 253
values: {1.0, 3.0, 5.0, 7.0, 9.0, 11.0, 13.0, 15.0, 17.0, 19.0, 21.0, 23.0, 25.0, 27.0, 29.0, 31.0, 33.0, 35.0, 37.0, 39.0, 41.0, 43.0, 45.0, 47.0, 49.0, 51.0, 53.0, 55.0, 57.0, 59.0, 61.0, 63.0, 65.0, 67.0, 69.0, 71.0, 73.0, 75.0, 77.0, 79.0, 81.0, 83.0, 85.0, 87.0, 89.0, 91.0, 93.0, 95.0, 97.0, 99.0, 101.0, 103.0, 105.0, 107.0, 109.0, 111.0, 113.0, 115.0, 117.0, 119.0, 121.0, 123.0, 125.0, 127.0, 129.0, 131.0, 133.0, 135.0, 137.0, 139.0, 141.0, 143.0, 145.0, 147.0, 149.0, 151.0, 153.0, 155.0, 157.0, 159.0, 161.0, 163.0, 165.0, 167.0, 169.0, 171.0, 173.0, 175.0, 177.0, 179.0, 181.0, 183.0, 185.0, 187.0, 189.0, 191.0, 193.0, 195.0, 197.0, 199.0, 201.0, 203.0, 205.0, 207.0, 209.0, 211.0, 213.0, 215.0, 217.0, 219.0, 221.0, 223.0, 225.0, 227.0, 229.0, 231.0, 233.0, 235.0, 237.0, 239.0, 241.0, 243.0, 245.0, 247.0, 249.0, 251.0, 253.0, 255.0, 257.0, 259.0, 2

In [5]:
# Remove country because there is only 1 valid value after pre-processing
cols = list(set(cols) - {"PAT_COUNTRY"})
df = df[cols]

# Convert some domains to integers
for att in ["TYPE_OF_ADMISSION", "PAT_ZIP", "PAT_COUNTY", "PAT_STATUS", "RACE", "ADMIT_WEEKDAY", "PAT_AGE"]:
    df[att] = df[att].astype(int)

In [6]:
# Select the subset of records with the 100 most frequent PRINC_DIAG_CODE
counts = df["PRINC_DIAG_CODE"].value_counts()
counts = counts.sort_values(ascending=False) # Sort descending
counts = counts.iloc[:100] # Select the first 100
most_freq_diag = counts.index.tolist()

# Filter the dataframe to rows stricted to the 100 most frequent PRINC_DIAG_CODE
df = df[df["PRINC_DIAG_CODE"].isin(most_freq_diag)]
print(f"Number of records after selecting 100 most frequent diagnosis: {len(df)}")

Number of records after selecting 100 most frequent diagnosis: 389172


In [7]:
# Removed negative charges
df = df[df["TOTAL_CHARGES"] > 0]
print(f"Number of records after removing negative: {len(df)}")

Number of records after removing negative: 389106


In [8]:
# Discretize TOTAL_CHARGES
def interval(value):
    interval_charges = [
        (0,1000),
        (1000,2000),
        (2000,3000),
        (3000,4000),
        (4000,5000),
        (5000,6000),
        (6000,7000),
        (7000,8000),
        (8000,9000),
        (9000,10000),
        (10000,15000),
        (15000,20000),
        (20000,25000),
        (25000,30000),
        (30000,40000),
        (40000,50000),
        (50000,100000),
        (100000,200000),
        (200000,500000),
        (500000,10**7)
    ]

    for left,right in interval_charges:
        if left <= value < right:
            if left == 500000:
                return "500000+"
            else:
                return f"[{left},{right})" # return interval

df["TOTAL_CHARGES"] = df["TOTAL_CHARGES"].apply(interval)
display(df)

,TOTAL_CHARGES,ADMIT_WEEKDAY,TYPE_OF_ADMISSION,SEX_CODE,PAT_AGE,PRINC_DIAG_CODE,PAT_ZIP,RACE,PAT_STATUS,PAT_COUNTY
1,"[50000,100000)",2,3,M,16,M1711,79912,4,6,141
2,"[50000,100000)",2,3,M,17,M1712,79928,5,6,141
3,"[40000,50000)",2,3,F,18,M1711,79835,5,6,141
4,"[30000,40000)",3,3,F,15,M1711,79901,5,6,141
5,"[40000,50000)",2,3,F,18,M1712,79936,5,6,141
...,...,...,...,...,...,...,...,...,...,...
788832,"[100000,200000)",2,2,F,4,Z5111,76116,4,1,439
788833,"[15000,20000)",6,1,M,2,J9601,75231,2,1,113
788834,"[30000,40000)",7,1,F,2,J9601,76048,4,1,221
788838,"[3000,4000)",5,4,F,0,Z3801,76273,4,1,181


In [9]:
# Save pre-processed file
df.to_csv(os.path.join(REPOSITORY_PATH, "data/hospitals/hospitals_pp.csv"), index=False, encoding="utf-8")